### Generate datapoints

In [ ]:
import numpy as np

m, n = 10, 10

silhouette = np.array([
    [0, 0, 0, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [1, 1, 1, 0, 0, 0, 1, 1, 1, 1],
    [1, 1, 0, 0, 0, 0, 0, 1, 1, 1],
    [1, 1, 1, 0, 0, 0, 1, 1, 1, 1],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
])

# 5 silhouettes per datapoint
# Same for all, for the sake of simplicity
gait_sequence = np.array([silhouette for _ in range(5)])

datapoints = []
for i in range(10):
    # Subject ID assigned to each
    subject_id = f'00{i%3 + 1}'
    # Same for all, for the sake of simplicity
    conditions = 'concrete,backpack,sneakers,1_week'
    # Create the datapoint as a structured array
    datapoint = np.array([(subject_id, conditions, gait_sequence)], 
                         dtype=[('subject_id', 'U10'), ('conditions', 'U50'), ('gait_sequence', 'O')])
    # Append the datapoint to our list
    datapoints.append(datapoint)

dataset = np.concatenate(datapoints)

# print(dataset)

### Initialization

In [ ]:
# Initial Matrix
U = np.eye(10)

# Parameters
Tmax_c, Tmax_d = (3, 3)

### CSA

In [ ]:
e = 3 # permissible error
for t in range(Tmax_c):
    # Step 2(a)
    F = []
    for datapoint in dataset:
        for silhouette in datapoint["gait_sequence"]:
            F += [U @ U.T @ silhouette]
    F = np.concatenate(F, axis=1)
    eigval, eigvec = np.linalg.eig(F @ F.T)
    eigvec = eigvec[:, np.argsort(eigval)[::-1]]
    V_new = eigvec[:, :5]
    # Step 2(b)
    G = []
    for datapoint in dataset:
        for silhouette in datapoint["gait_sequence"]:
            G += [silhouette @ V_new @ V_new.T]
    G = np.concatenate(G, axis=1)
    eigval, eigvec = np.linalg.eig(G @ G.T)
    eigvec = eigvec[:, np.argsort(eigval)[::-1]]
    U_new = eigvec[:, :5]
    # Step 2(c)
    if t > 1 and np.linalg.norm(U_new - U, "fro") < 10 * e and np.linalg.norm(V_new - V, "fro") < 10 * e:
        break
    U, V = U_new, V_new

# Step 3
